In [ ]:
# Copyright 2024 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Deploy a model on Vertex AI

### Configuration

In [2]:
! pip install --upgrade --quiet  google-cloud-aiplatform \
                                 google-cloud-storage

In [4]:
# Enable Artifact Registry
! gcloud services enable artifactregistry.googleapis.com

In [13]:
PROJECT_ID = "ai-hangsik" 
LOCATION = "us-central1" 
BUCKET_URI = f"gs://sllm_0116" 

# REPOSITORY = "iris-custom-container-repo"

MODEL_ARTIFACT_DIR = "custom-container-prediction-model"
REPOSITORY = "custom-container-prediction"
IMAGE = "sklearn-fastapi-server"
MODEL_DISPLAY_NAME = "sklearn-custom-container"


In [19]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

In [14]:
# Create artifacts repository

! gcloud artifacts repositories create $REPOSITORY --repository-format=docker --location={LOCATION} --description="Docker repository for Customer container"

Create request issued for: [custom-container-prediction]
Waiting for operation [projects/ai-hangsik/locations/us-central1/operations/a1c
f7032-336e-407c-8fac-ecc8c0fe6ae9] to complete...done.                         
Created repository [custom-container-prediction].


In [16]:
! gcloud artifacts repositories list

Listing items under project ai-hangsik, across all locations.

                                                                                          ARTIFACT_REGISTRY
REPOSITORY                     FORMAT  MODE                 DESCRIPTION                               LOCATION         LABELS  ENCRYPTION          CREATE_TIME          UPDATE_TIME          SIZE (MB)
cloud-run-source-deploy        DOCKER  STANDARD_REPOSITORY  Cloud Run Source Deployments              asia-northeast3          Google-managed key  2024-03-01T14:59:17  2024-03-01T23:38:33  1505.522
kubeflow-test                  DOCKER  STANDARD_REPOSITORY                                            asia-northeast3          Google-managed key  2024-11-10T07:54:48  2024-11-10T09:23:27  604.943
custom-container-prediction    DOCKER  STANDARD_REPOSITORY  Docker repository for Customer container  us-central1              Google-managed key  2025-02-03T09:10:17  2025-02-03T09:10:17  0
custom-inference-gpu           DOCKER  STAND

In [17]:
!gcloud builds submit --region={LOCATION} --tag={LOCATION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}

Creating temporary archive of 14 file(s) totalling 200.7 KiB before compression.
Uploading tarball of [.] to [gs://ai-hangsik_cloudbuild/source/1738573970.95662-454afa4139494d609c25de3006e5997f.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/ai-hangsik/locations/us-central1/builds/eb29a334-8512-4eee-b97f-cc84e3a00e3f].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds;region=us-central1/eb29a334-8512-4eee-b97f-cc84e3a00e3f?project=721521243942 ].
Waiting for build to complete. Polling interval: 1 second(s).
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "eb29a334-8512-4eee-b97f-cc84e3a00e3f"

FETCHSOURCE
Fetching storage object: gs://ai-hangsik_cloudbuild/source/1738573970.95662-454afa4139494d609c25de3006e5997f.tgz#1738573971309841
Copying gs://ai-hangsik_cloudbuild/source/1738573970.95662-454afa4139494d609c25de3006e5997f.tgz#1738573971309841...
/ [1 files][ 36.0 KiB/ 36.0 KiB]                       

In [20]:

model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=f"{BUCKET_URI}/{MODEL_ARTIFACT_DIR}",
    serving_container_image_uri=f"{LOCATION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}",
)

Creating Model
Create Model backing LRO: projects/721521243942/locations/us-central1/models/8814543926833709056/operations/3115112051479937024
Model created. Resource name: projects/721521243942/locations/us-central1/models/8814543926833709056@1
To use this Model in another session:
model = aiplatform.Model('projects/721521243942/locations/us-central1/models/8814543926833709056@1')


In [21]:
deployed_model = model.deploy(
    endpoint=aiplatform.Endpoint.create(display_name="custom-prediction-endpoint"),
    machine_type="n1-standard-4"
)

Creating Endpoint
Create Endpoint backing LRO: projects/721521243942/locations/us-central1/endpoints/4684087759604809728/operations/657976242284265472
Endpoint created. Resource name: projects/721521243942/locations/us-central1/endpoints/4684087759604809728
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/721521243942/locations/us-central1/endpoints/4684087759604809728')
Deploying model to Endpoint : projects/721521243942/locations/us-central1/endpoints/4684087759604809728
Deploy Endpoint model backing LRO: projects/721521243942/locations/us-central1/endpoints/4684087759604809728/operations/6277342677335801856
Endpoint model deployed. Resource name: projects/721521243942/locations/us-central1/endpoints/4684087759604809728


In [22]:
# Send some sample data to the endpoint
deployed_model.predict(instances=[[6.7, 3.1, 4.7, 1.5], [4.6, 3.1, 1.5, 0.2]])

Prediction(predictions=['versicolor', 'setosa'], deployed_model_id='1889133599844728832', metadata=None, model_version_id='1', model_resource_name='projects/721521243942/locations/us-central1/models/8814543926833709056', explanations=None)

In [24]:
# Fetch the endpoint name
ENDPOINT_ID = deployed_model.name

In [25]:
# Send a prediction request using sample data
! curl \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-d @instances.json \
https://{LOCATION}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{ENDPOINT_ID}:predict

{
  "predictions": [
    "versicolor",
    "setosa"
  ],
  "deployedModelId": "1889133599844728832",
  "model": "projects/721521243942/locations/us-central1/models/8814543926833709056",
  "modelDisplayName": "sklearn-custom-container",
  "modelVersionId": "1"
}
